In [ ]:
%run generator/statementGenerator.ipynb
%run generator/IDsPlayerTimeGenerator.ipynb
%run generator/TypeChecker.ipynb

**Select number of traces, players and IDs** to generate

In [ ]:
nbtracesGameObjects = widgets.IntSlider(description="Traces", min=0, value=1, max=10000, continuous_update=False)
nbtracesGameObjectstext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbtracesGameObjects, 'value'), (nbtracesGameObjectstext, 'value'))
tracesGameObjects=VBox([nbtracesGameObjects,nbtracesGameObjectstext])

nbplayersGameObjects = widgets.IntSlider(description="Players", min=0, value=1, continuous_update=False)
nbplayersGameObjectstext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbplayersGameObjects, 'value'), (nbplayersGameObjectstext, 'value'))
widgets.link((nbtracesGameObjects, 'value'), (nbplayersGameObjects, 'max'))
playersGameObjects=VBox([nbplayersGameObjects,nbplayersGameObjectstext])

nbIDsGameObjects = widgets.IntSlider(description="IDs", min=0, value=1, continuous_update=False)
nbIDsGameObjectstext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbIDsGameObjects, 'value'), (nbIDsGameObjectstext, 'value'))
widgets.link((nbtracesGameObjects, 'value'),(nbIDsGameObjects, 'max'))
IDsGameObjects=VBox([nbIDsGameObjects,nbIDsGameObjectstext])

vbox_nb_traces_ID_player_game_objects=VBox([tracesGameObjects,playersGameObjects, IDsGameObjects])

**GameObjects**

In [ ]:
verbs_gameObjects=["interacted", "used"]
checkboxes_verbs_gameObjects=GeneratorMultiCheckboxWidget("GameObjects verbs", verbs_gameObjects)

types_gameObjects=["enemy", "npc", "item", "gameobject"]
checkboxes_types_gameObjects=GeneratorMultiCheckboxWidget("GameObjects types", types_gameObjects)

action_types=["talk_to", "examine", "use", "drag_to", "custom"]
checkboxes_action_types=GeneratorMultiCheckboxWidget("ActionTypes", action_types)

template_result_action_type_interact={
    "extensions": {
        "action_type": None,
    }
}

def generate_GameObjects(change=None):
    start_time=datetime.now()
    if change!=None:
        with output:
            print("Starting the data generation")
    verbs=checkboxes_verbs_gameObjects.get_selected_options()
    types=checkboxes_types_gameObjects.get_selected_options()
    actionTypes=checkboxes_action_types.get_selected_options()
    ids=generate_IDs(nbIDsGameObjects.value)
    times=generate_times_datas(nbtracesGameObjects.value)
    player=generate_players(nbplayersGameObjects.value)
    statements=[]
    checkTypes={}
    checkActionType={}
    for i in range(nbtracesGameObjects.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomVerb=random.choice(verbs)
        randomType= checkType(checkTypes,randomID,randomPlayer,types)
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomVerb=="interacted":
            if randomType not in checkActionType.keys():
                checkActionType[randomID]=random.choice([True, False])
            generateActionType=checkActionType[randomID]
            if generateActionType and not len(actionTypes)==0:
                actiontype=random.choice(actionTypes)
                if actiontype=="custom":
                    actiontype=str(uuid.uuid4())
                    actionTypes.append(actiontype)
                result_action_type=copy.deepcopy(template_result_action_type_interact)
                result_action_type["extensions"]["action_type"]=actiontype
                statement["result"]=result_action_type
        statements.append(statement)
    if change==None:
        return statements
    else:
        save_generated_data(statements, verbs, types)
        end_time=datetime.now()
        delta=end_time-start_time
        with output:
            print("End the data generation")
            print("Generated in "+str(delta))
            clear_output(wait=True)

buttonGameObjects=widgets.Button(description='Generate Game Objetcs Data', layout = widgets.Layout( width='auto'))
buttonGameObjects.on_click(generate_GameObjects)

global tabgameObjects
tabgameObjects=VBox([
    HBox([
        checkboxes_verbs_gameObjects.multi_select,
        checkboxes_types_gameObjects.multi_select,
        checkboxes_action_types.multi_select
    ]),
    HBox([
        vbox_nb_traces_ID_player_game_objects
    ]),
    buttonGameObjects
])